In [1]:
!git clone https://github.com/ruzanovad/diploma.git

Cloning into 'diploma'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (427/427), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 427 (delta 236), reused 334 (delta 143), pack-reused 0 (from 0)
Receiving objects: 100% (427/427), 28.95 MiB | 40.77 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [2]:
%cd diploma/cnn-rnn

/kaggle/working/diploma/cnn-rnn


In [3]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

In [5]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

In [ ]:
%run main.py --batch-size 8 --accumulate-batch 64 --predict --log-step 1 \ --num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed \
--data-path /kaggle/input/im2latex100k \
--vocab_file data/vocab/latex_tokens.json
--predict-img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png

ImportError: cannot import name 'LatexDataset' from 'data.dataset' (/kaggle/working/diploma/cnn-rnn/data/dataset.py)

In [ ]:
%run main.py --batch-size 8 --accumulate-batch 64 --predict --log-step 1 \
--train-sample 0 --test-sample 0 --val-sample 0 --num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png